In [1]:
# import dependencies
import pandas as pd
import numpy as np

In [2]:
# raw data file
file_to_load = "Resources/purchase_data.csv"

# read purchasing file and store into pandas data frame
purchase_data_df = pd.read_csv(file_to_load)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
# calculate total number of players
count_players = len(purchase_data_df["SN"].value_counts())
player_dict = [{"PLAYERS TOTAL": count_players}]
player_df = pd.DataFrame(player_dict)
player_df

,PLAYERS TOTAL
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
def purchasing_total():
    # calculate number of unique items
    count_unique = len(purchase_data_df["Item ID"].unique())
    
    # calculate number of purchases
    purchase_count = purchase_data_df["SN"].value_counts()
    purchase_total = purchase_count.sum()

    # calculate total revenue
    total_revenue = purchase_data_df["Price"].sum()

    # calculate average price
    avr_price = round(total_revenue/purchase_total, 2)
    
    # put metrics into a dataframe 
    summary_table = pd.DataFrame({
                                    "UNIQUE ITEMS": [count_unique],
                                    "AVG PRICE": [avr_price],
                                    "NUMBER OF PURCHAES": [purchase_total],
                                    "TOTAL REVENUE": [total_revenue]
    })
    return summary_table

analysis_total_df = purchasing_total()

# format dataframe
analysis_total_df.style.format({'AVERAGE PRICE': '${:.2f}', 'TOTAL REVENUE': '${:,.2f}'})

,UNIQUE ITEMS,AVG PRICE,NUMBER OF PURCHAES,TOTAL REVENUE
0,183,3.05,780,"$2,379.77"


## Gender Demographics

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
def gender_demographics():
    # determine unique values in gender column
    unique_gender = purchase_data_df["Gender"].unique()

    # drop duplicates
    no_duplicates = purchase_data_df.drop_duplicates(['SN'], keep ='last')

    # calculate total number in gender column
    gender_total = len(no_duplicates["Gender"])

    # MALE players calculations
    male_df = no_duplicates.loc[purchase_data_df["Gender"] == "Male"]
    total_male = len(male_df["Gender"])
    percent_male = round(total_male / gender_total * 100,2)

    # FEMALE players calculations
    female_df = no_duplicates.loc[purchase_data_df["Gender"] == "Female"]
    total_female = len(female_df)
    percent_female = round(total_female / gender_total * 100,2)

    # OTHER/NON-DISCLOSED players calculations
    other_df = no_duplicates.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed"]
    total_other = len(other_df)
    percent_other = round(total_other / gender_total * 100,2)

    # put metrics into a dataframe 
    summary_table = pd.DataFrame({
                                    "GENDER": ["MALE", "FEMALE", "OTHER / NON-DISCLOSED"],                
                                    "TOTAL": [total_male, total_female, total_other],
                                    "% OF PLAYERS": [percent_male, percent_female, percent_other]
    })
    return summary_table

analysis_gender_df = gender_demographics()

# format dataframe
analysis_gender_df = analysis_gender_df.set_index("GENDER")
analysis_gender_df.style.format({"% OF PLAYERS": "{:.2f} %"})

,TOTAL,% OF PLAYERS
GENDER,,
MALE,484,84.03 %
FEMALE,81,14.06 %
OTHER / NON-DISCLOSED,11,1.91 %



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, etc. by gender


* For normalized purchasing, divide total purchase value by purchase count, by gender


* Create a summary data frame to hold the results


* Give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
def purchasing_analysis_gender():
    # drop duplicates
    no_duplicates = purchase_data_df.drop_duplicates(['SN'], keep ='last')
    
    # calculate total purchase count per gender
    purchase_count = (purchase_data_df.groupby('Gender')['Gender'].count())

    # calculate average purchase price per gender
    purchase_average = (purchase_data_df.groupby('Gender')['Price'].mean())

    # calculate total purchase value per gender
    purchase_total = (purchase_data_df.groupby('Gender')['Price'].sum())

    # calculate normalized purchase count per gender
    normalized_values = no_duplicates["Gender"].value_counts()

    # calculate normalized purchase price per gender
    avr_norm = round(purchase_total / normalized_values, 2)

    # format summary table
    purchase_count = purchase_count.map("{:,}".format)
    purchase_total = purchase_total.map("${:,.2f}".format)
    purchase_average = purchase_average.map("${:,.2f}".format)
    avr_norm = avr_norm.map("${:,.2f}".format)

    # put metrics into a dataframe 
    summary_table = pd.DataFrame({
                                "PURCHASE COUNT": purchase_count,
                                "AVG PURCHASE PRICE": purchase_average,
                                "TOTAL PURCHASE VALUE": purchase_total,
                                "NORMALIZED AVG PRICE": avr_norm
    })
    return summary_table
    
purchasing_analysis_gender = purchasing_analysis_gender()
purchasing_analysis_gender

,PURCHASE COUNT,AVG PURCHASE PRICE,TOTAL PURCHASE VALUE,NORMALIZED AVG PRICE
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
def age_demographics():
    # establish bins for ages
    age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
    group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

    # place the data series into a new column inside of the DataFrame
    purchase_data_df["Age Group"] = pd.cut(purchase_data_df["Age"], age_bins, labels=group_names)

    # calculate purchase count per age group
    purchase_count = (purchase_data_df.groupby('Age Group')['Age'].count())

    # calculate average purchase price
    purchase_average = (purchase_data_df.groupby('Age Group')['Price'].mean())

    # calculate total purchase value per age
    purchase_total = (purchase_data_df.groupby('Age Group')['Price'].sum())

    # create data frame of unique player names by only keeping the last occurance
    no_duplicates = purchase_data_df.drop_duplicates(['SN'], keep ='last')

    # calculate unduplicated purchase count per age group
    no_duplicates_purchase_count = (no_duplicates.groupby('Age Group')['Age'].count())

    # calculate normalized average purchase price per age group
    avr_norm = round(purchase_total / no_duplicates_purchase_count, 2)

    # format summary table
    purchase_count = purchase_count.map("{:,}".format)
    purchase_average = purchase_average.map("${:,.2f}".format)
    purchase_total = purchase_total.map("${:,.2f}".format)
    avr_norm = avr_norm.map("${:,.2f}".format)

    # put metrics into a dataframe
    summary_table = pd.DataFrame({
                                "PURCHASE COUNT": purchase_count,
                                "AVERAGE PURCHASE PRICE": purchase_average,
                                "TOTAL PURCHASE VALUE": purchase_total,
                                "NORMALIZED AVERAGE PRICE": avr_norm
    })
    return summary_table


age_demographics = age_demographics()
age_demographics

,PURCHASE COUNT,AVERAGE PURCHASE PRICE,TOTAL PURCHASE VALUE,NORMALIZED AVERAGE PRICE
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
def top_spenders():
    # calculate total value per player
    purchase_total = (purchase_data_df.groupby('SN')['Price'].sum())

    # calculate total purchase count per player
    purchase_count = (purchase_data_df.groupby('SN')['Item ID'].count())

    # calculate average purchase price per player
    purchase_average = (purchase_data_df.groupby('SN')['Price'].mean())

    # format summary table
    purchase_count = purchase_count.map("{:,}".format)
    purchase_average = purchase_average.map("${:,.2f}".format)
    purchase_total = purchase_total.map("${:,.2f}".format)

    # put metrics into a dataframe
    summary_table = pd.DataFrame({
                                "PURCHASE COUNT": purchase_count,
                                "AVG PURCHASE PRICE": purchase_average,
                                "TOTAL PURCHASE VALUE": purchase_total
    })

    # sort from highest purchase value to lowest
    summary_table.sort_values('PURCHASE COUNT', ascending = False, inplace=True)
    
    return summary_table

top_spenders = top_spenders()
top_spenders.head()

,PURCHASE COUNT,AVG PURCHASE PRICE,TOTAL PURCHASE VALUE
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
def popular_items():
    # calculate total purchase count per item name and ID number
    purchase_count = (purchase_data_df.groupby(["Item ID", "Item Name"])['Item ID'].count())

    # calculate total value per item name and ID number
    purchase_total = (purchase_data_df.groupby(["Item ID", "Item Name"])['Price'].sum())

    # calculate average purchase price per item name and ID number
    purchase_average = (purchase_data_df.groupby(["Item ID", "Item Name"])['Price'].mean())

    # format summary table
    purchase_count = purchase_count.map("{:,}".format)
    purchase_average = purchase_average.map("${:,.2f}".format)
    purchase_total = purchase_total.map("${:,.2f}".format)

    # put metrics into a dataframe
    summary_table = pd.DataFrame({
                                "PURCHASE COUNT": purchase_count,
                                "AVERAGE PURCHASE PRICE": purchase_average,
                                "TOTAL PURCHASE VALUE": purchase_total
    })

    # sort from highest purchase value to lowest
    summary_table.sort_values('PURCHASE COUNT', ascending = False, inplace=True)
    
    return summary_table

popular_items = popular_items()
popular_items.head()

,,PURCHASE COUNT,AVERAGE PURCHASE PRICE,TOTAL PURCHASE VALUE
Item ID,Item Name,,,
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
92,Final Critic,8,$4.88,$39.04
75,Brutality Ivory Warmace,8,$2.42,$19.36


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
def profitable_items():
    # calculate total purchase count per item name and ID number
    purchase_count = (purchase_data_df.groupby(["Item ID", "Item Name"])['Item ID'].count())

    # calculate total value per item name and ID number
    purchase_total = (purchase_data_df.groupby(["Item ID", "Item Name"])['Price'].sum())

    # calculate average purchase price per item name and ID number
    purchase_price = (purchase_data_df.groupby(["Item ID", "Item Name"])['Price'].mean())

    # format summary table
    purchase_count = purchase_count.map("{:,}".format)
    purchase_price = purchase_price.map("${:,.2f}".format)
    purchase_total = purchase_total.map("${:,.2f}".format)

    # put metrics into a dataframe
    summary_table = pd.DataFrame({
                                "PURCHASE COUNT": purchase_count,
                                "ITEM PRICE": purchase_price,
                                "TOTAL PURCHASE VALUE": purchase_total
    })

    # sort from highest purchase value to lowest
    summary_table.sort_values('PURCHASE COUNT', ascending = False, inplace=True)
    
    return summary_table

profitable_items = profitable_items()
profitable_items.head()

,,PURCHASE COUNT,ITEM PRICE,TOTAL PURCHASE VALUE
Item ID,Item Name,,,
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
92,Final Critic,8,$4.88,$39.04
75,Brutality Ivory Warmace,8,$2.42,$19.36


## Trend Analysis 

- Of the 576 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

- Our peak age demographic falls between 20-24 (47%) with secondary groups falling between 15-19 (17%) and 25-29 (13%).

- Most active users purchase from 3 to 5 games.

- Top choice games are "Oathbreaker, Last Hope of the Breaking Storm" (purchased by 12 users), "Nirvana" (purchased by 9 users), "Fiery Glass Crusader" (purchased by 9 users), "Final Critic" (purchased by 8 users), and "Singed Scalpel" (purchased by 8 users)